In [68]:
# import libraries

from bs4 import BeautifulSoup
import re
import requests
import time
import smtplib
import csv
import datetime
from dataclasses import dataclass

In [109]:
# Connect to Website & Scrape Data

@dataclass
class Job:
    title: str
    company: str
    location: str
    date_posted: str

// Request page information from URL
def requestPage(keywords: str, page: int):
    url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?geoId=106518625&keywords=%s&start=%s" % (keywords, page*25) 
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
    requests.get(url, headers=headers,cookies={'__hs_opt_out': 'no'})
    page = requests.get(url, headers=headers)
    return BeautifulSoup(page.content, 'html.parser')

// Find all job divs
def findJobs(soup: BeautifulSoup):
    jobs = soup.find_all(["div","a"],class_="job-search-card")
    return jobs

// Parse jobs for information
def parseJob(job: BeautifulSoup):
    info = job.find("div",class_="base-search-card__info")
    title = info.h3.text.strip()
    company = info.h4.text.strip()
    location = info.div.span.text.strip()
    date_posted = info.div.time["datetime"]
    return Job(title, company, location, date_posted)

// Start webscrape and write data to CSV
with open('LinkedInWebScraperDataset.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(['Title', 'Company', 'Location','Date_Posted', 'Date_Collected'])

    today = datetime.date.today()
    page = 0
    while True:
        soup = requestPage("data%20analyst", page)  
        jobs = findJobs(soup)
        if len(jobs) == 0:
            break
        for job in jobs:
            data = parseJob(job)
            writer.writerow([data.title, data.company, data.location, data.date_posted, today])
        page+= 1